In [ ]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    stop_words = fp.read()

In [ ]:
import numpy as np # linear algebra
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:
similarity_document = ctrl.Antecedent(np.arange(0, 1.25, .1), 'similarity')
senlength = ctrl.Antecedent(np.arange(0, 1.25, .1), 'senlength')
senpos=ctrl.Antecedent(np.arange(0, 2, .1), 'senpos')
nounplace=ctrl.Antecedent(np.arange(0, 1.2, .1), 'nounplace')
pronounplace=ctrl.Antecedent(np.arange(0, 1.2, .1), 'pronounplace')
tfidfscore = ctrl.Antecedent(np.arange(0, 1.25, .1), 'tfidfscore')


#consequent
rank = ctrl.Consequent(np.arange(0, 24, 1), 'rank')

In [ ]:
similarity_document.automf(3)
senlength.automf(3)
senpos.automf(3)
nounplace.automf(3)
pronounplace.automf(3)
tfidfscore.automf(3)

In [ ]:
#consequent
rank['low']=fuzz.trimf(rank.universe,[0,0,10])
rank['medium']=fuzz.trimf(rank.universe,[0,10,24])
rank['high']=fuzz.trimf(rank.universe,[10,24,24])

In [ ]:
rule1 = ctrl.Rule(similarity_document['poor'] & senlength['poor'] & senpos['poor'] & nounplace['poor'] & pronounplace['poor'] & tfidfscore['poor']  , rank['low'])
rule2 = ctrl.Rule(similarity_document['average'] & senlength['average'] & senpos['average'] & nounplace['average'] & pronounplace['average'] & tfidfscore['average'], rank['medium'])
rule3 = ctrl.Rule(similarity_document['good'] & senlength['good'] & senpos['good']& nounplace['good'] & pronounplace['good'] & tfidfscore['good'] , rank['high'])

In [ ]:
rankFIS = ctrl.ControlSystem([rule1,rule2,rule3])
rankFIS = ctrl.ControlSystemSimulation(rankFIS)

In [ ]:
from nltk.tokenize import word_tokenize
def tokenizefortfidf(originalsen):
    tokenized_sents = [word_tokenize(i) for i in originalsen]
    sen_list1=[]
    for i in tokenized_sents:
        sen_list1=sen_list1+i
    return sen_list1
#print(sen_list1)

In [ ]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(list(zip(wordlist,wordfreq)))

In [ ]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [ ]:
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)

In [ ]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    for word, val in docList.items():
        idfDict[word] = math.log10(N / float(val) + 1)
        
    return(idfDict)

In [ ]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)

In [ ]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [ ]:
lst=[]
def loopsecond(word1len,word1,idf,j):
    for k in range(word1len): 
            if idf.index[j] == word1[k]:
                lst.append(idf['ratio'][j])
                break

def loopfirst(leng,word1,idf):
    word1len = len(word1)
    for j in range(leng):
        #print(j)
        loopsecond(word1len,word1,idf,j)

In [ ]:
import nltk
nltk.download('indian')
nltk.download('punkt')

In [ ]:
import nltk
from nltk.tag import tnt 
from nltk.corpus import indian 

#train_data = indian.tagged_sents('gu.pos')
train_data = indian.tagged_sents('C:/Users/HM-AM/Documents/Jupyter Notebooks/gu.pos')
tnt_pos_tagger = tnt.TnT() 
tnt_pos_tagger.train(train_data)

In [ ]:
size = int(len(train_data) * 0.9)

In [ ]:
#train_sents = train_data
train_sents = train_data[:size]
test_sents = train_data[size:]

In [ ]:
def backoff_tagger(train_sentences, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sentences, backoff=backoff)
        
    return backoff

In [ ]:
back_tagger = nltk.DefaultTagger('NN')
Combine_tagger = backoff_tagger(train_sents,
[nltk.UnigramTagger, nltk.BigramTagger, nltk.TrigramTagger], backoff = back_tagger)

Combine_tagger.evaluate(test_sents)

In [ ]:
def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

In [ ]:
from statistics import mean
def Average(lst):
    return mean(lst)

In [ ]:
from rouge.rouge import rouge_n_sentence_level
from rouge import rouge_n_summary_level
from rouge import rouge_l_summary_level
from rouge import rouge_w_sentence_level
from rouge import rouge_w_summary_level
import os
from urllib.parse import urlparse
def comparerouge1(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a1_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_1 = rouge_n_summary_level(tokenize_words, tokenize_words1, 1)
    #rouge1.append(rouge_1)
    #print('ROUGE-1: %f' % rouge_1)
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
    #print('ROUGE-2: %f' % rouge_2)
    #rouge2.append(rouge_2)
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    #print('ROUGE-L: %f' % rouge_l)
    #rougel.append(rouge_l)
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    #avg=(rouge_1+rouge_2+rouge_l+rouge_w)/4
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_1

In [ ]:
def comparerouge2(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a1_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
   
    _, _, rouge_2 = rouge_n_summary_level(tokenize_words, tokenize_words1, 2)
   
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_2   

In [ ]:
def comparerougel(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a1_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    
    _, _, rouge_l = rouge_l_summary_level(tokenize_words, tokenize_words1)
    
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_l    

In [ ]:
def comparerougew(reference,originalfull_str,filename):
    a = urlparse(filename)
    tokenize_words=[]
    tokenize_words1=[]
    for tokenize in indic_tokenize.trivial_tokenize(originalfull_str):
            tokenize_words.append(tokenize)
    sentences=sentence_tokenize.sentence_split(originalfull_str, lang='gu')
    with codecs.open(reference+"a1_"+os.path.basename(a.path), encoding='utf-8') as f1:
        data1 = f1.read()
    for tokenize1 in indic_tokenize.trivial_tokenize(data1):
            tokenize_words1.append(tokenize1)
        # Convert list of strings to string
    originalfull_str1 = convert_list_to_string(tokenize_words1)
    sentences1=sentence_tokenize.sentence_split(originalfull_str1, lang='gu')
    _, _, rouge_w = rouge_w_summary_level(tokenize_words, tokenize_words1)
    
    tokenize_words.clear()
    tokenize_words1.clear()
    return rouge_w

In [ ]:
with open('H:/College WOrk/papers/stop-words.txt',encoding='utf-16') as fp:
    v = fp.read()

def removestopwords(line):
    #print("original line",line)
    querywords = line.split()

    resultwords  = [word for word in querywords if word.lower() not in v]
    result = ' '.join(resultwords)
    #print(result)
    return result

In [ ]:
import codecs
import string
import re
from bs4 import BeautifulSoup  
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize
from difflib import SequenceMatcher
import pandas as pd
import math
import io

nounlist=['N-NNP','N_NN','N_NNV','N_NST','NN']
pronounlst=['PR_PRC','PR_PRF','PR_PRI','PR_PRP','PR_PRQ']
fc=[]
li=[]
maintext=[]
rouge1=[]
rouge2=[]
rougel=[]
rougew=[]
class ShortText:
    def __init__(self, my_id, human_label, summary, short_text):
        self.id = my_id         
        self.human_label = human_label    
        self.summary = summary 
        self.short_text = short_text
    def __str__(self):
        '''
        For printing purposes.
        '''
        return '%d\t%d\t%s\t%s' % (self.id, self.human_label, self.summary, self.short_text)

def load_file8(filename,createfile):
    tokenize_words=[]
    titlesentences=[]
    file1 = open(createfile, "w", encoding="utf-8")
    #retrieve the original text 
    with codecs.open(filename, encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    docno=soup.find_all('docno')
    text=soup.find_all('text')
    titleno = soup.find_all('title')
    instances = {}
    my_id = 0
    for n,tit,maintitle in zip(docno,text,titleno):
        print("This is doc file",n)
        count=0
        tit=tit.get_text()
        tit=str(tit)
        #print("this is text",tit)
        maintitle=maintitle.get_text()
        maintitle=str(maintitle)
        maintitlelist=[]
        for tokenize in indic_tokenize.trivial_tokenize(maintitle):
            titlesentences.append(tokenize)
        list8 = [''.join(c for c in s if c not in string.punctuation) for s in titlesentences]
        list8 = [s for s in list8 if s]
        originalsen=[]
        sentences=sentence_tokenize.sentence_split(tit, lang='gu')
        totalsen=len(sentences)
        #print(sentences)
        list3 = [''.join(c for c in s if c not in string.punctuation) for s in sentences]
        list3 = [s for s in list3 if s]
        #print("this is list3",list3)
        dictionary = wordListToFreqDict(list3)
        sorteddict = sortFreqDict(dictionary)
        context=listToString(list3)
        wordDictA = dict.fromkeys(list3, 0)
        for word in list3:
            wordDictA[word]+=1
        df2=pd.DataFrame([wordDictA])
        tfFirst = computeTF(wordDictA, context)
        tf = pd.DataFrame([tfFirst])
        idfs = computeIDF(wordDictA)
        idfFirst = computeTFIDF(tfFirst, idfs)
        #idfSecond = computeTFIDF(tfSecond, idfs)
        #putting it in a dataframe
        idf= pd.DataFrame([idfFirst])
        idf = idf.transpose()
        idf.columns=['ratio']
        final_df = pd.DataFrame(columns=['sentence', 'score'])
        for i in range(len(list3)):
            list3[i]=str(list3[i])[1:-1]
            word = listToString(list3[i])
   
            word1 = word.split()
    
            lenofdf=len(idf)
            loopfirst(lenofdf,word1,idf)
            total = sum(lst)+1
            final_df = final_df.append({'sentence': list3[i], 'score': total},ignore_index='True')
    
            lst.clear()
        #print("this is final_df",final_df)
        lengthoflongestsen=len(max(sentences, key=len))
        #print(lengthoflongestsen)
        #list3 = [s for s in sentences if s]
        df_fuzzy = pd.DataFrame(columns=['sentence', 'score'])
        for index, t in enumerate(list3):
            nounplace=0
            pronounplace=0
            #print("this is docno",n)
            sen=removestopwords(t)
            res_list = [SequenceMatcher(None, sen, list3[i]).ratio() for i in range(len(list3)) if i!=index]
            average = Average(res_list)
            rankFIS.input['similarity'] = average            
            senlength=len(t)/lengthoflongestsen
            rankFIS.input['senlength']=senlength
            
            if index==0:
                senpos=0+0.1
            elif index==len(sentences)-1:
                senpos=1
            else:
                senpos=index/totalsen
            #print("this is senpos",senpos)
            rankFIS.input['senpos']=senpos
            
            tagged_words = (Combine_tagger.tag(nltk.word_tokenize(t)))
            lst1=[]
            for w in tagged_words:
                lst1.append(w[1])
            for w in lst1:
                if w in nounlist:
                    nounplace=1
            #print("this is nounplace",nounplace)
            rankFIS.input['nounplace']=nounplace
            
            for w1 in lst1:
                if w1 in pronounlst:
                    pronounplace=1          
            #print("this is pronounplace",pronounplace)
            rankFIS.input['pronounplace']=pronounplace+0.1
            #print(final_df.loc[index]['score'])
            rankFIS.input['tfidfscore']=final_df.loc[index]['score']
            rankFIS.compute()
#             print ("the answer is")
#             print (rankFIS.output['rank']) 
            df_fuzzy = df_fuzzy.append({'sentence': t, 'score': rankFIS.output['rank']},ignore_index='True')
        df_fuzzy.sort_values(by=['score'], inplace=True, ascending=False)
        df_fuzzy2=df_fuzzy['sentence'].head(round(20*len(df_fuzzy.index)/100))
        listtoretrieve=list(df_fuzzy2.index.values) 
        listtoretrieve.sort()
        df2 = pd.DataFrame(list3,columns=['sentences'])
        new_list = df2.values.tolist()
        res1=[]

        for x in new_list:
            res1=res1+x
        finaldraft=""
        for i in listtoretrieve:
            finaldraft+=res1[i]
        finaldraft=" ".join(finaldraft.split())
        rouge_1=comparerouge1("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/sports/annotator1/",finaldraft,filename)
        rouge_2=comparerouge2("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/sports/annotator1/",finaldraft,filename)
        rouge_l=comparerougel("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/sports/annotator1/",finaldraft,filename)
        rouge_w=comparerougew("C:/Users/HM-AM/Documents/GitHub/Gujarati-Textsummarization-corpus/gujarati Text summarization dataset/sports/annotator1/",finaldraft,filename)
        
        rouge1.append(rouge_1)
        rouge2.append(rouge_2)
        rougel.append(rouge_l)
        rougew.append(rouge_w)
        
        rouge1average=Average(rouge1)
        rouge2average=Average(rouge2)
        rougelaverage=Average(rougel)
        rougewaverage=Average(rougew)
        print("rouge1 average",rouge1average)
        print("rouge2 average",rouge2average)
        print("rougel average",rougelaverage)
        print("rougew average",rougewaverage)
        #print(finaldraft)
        with io.open(createfile, "w", encoding="utf-8") as f:
            f.write(finaldraft)
        instances = {}
    
        
    return instances

In [ ]:
import os

#for root, dirs, files in os.walk("/home/harsh/Documents/gujdocs/gsf2003/"):
for root, dirs, files in os.walk("G:/Movies/gujarati Text summarization dataset/sports/"):    
#for root, dirs, files in os.walk("G:/Movies/post_1/"):    
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(root, file), 'r') as f:
                #print(os.path.join(root, file))
                load_file8(os.path.join(root, file),"G:/Movies/gujarati Text summarization datasetfuzzy/sports/"+file)